## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 4: Adult
### (Missing values: yes)

In [88]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV

In [89]:
data = pd.read_csv('./Datasets/adult.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Age            32561 non-null  int64 
 1   WorkClass      30725 non-null  object
 2   Fnlwgt         32561 non-null  int64 
 3   Education      32561 non-null  object
 4   EducationNum   32561 non-null  int64 
 5   MaritalStatus  32561 non-null  object
 6   Occupation     30718 non-null  object
 7   Relationship   32561 non-null  object
 8   Race           32561 non-null  object
 9   Sex            32561 non-null  object
 10  CapitalGain    32561 non-null  int64 
 11  CapitalLoss    32561 non-null  int64 
 12  HoursPerWeek   32561 non-null  int64 
 13  NativeCountry  31978 non-null  object
 14  Income         32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [90]:
# Missing values imputation
"""
Observam ca avem missing values in 3 dintre atribute:
- WorkClass
- Occupation
- NativeCountry

Din acest motiv vom folosi un fillna pentru calcularea valorilor ce ar putea substitui missing values. Vom crea o noua clasa unknown pentru fiecare valoare lipsa. Nu alegem alta metoda de fill, precum: cea mai itnalnita valoare, medie, mediana, etc. deoarece umplerea atributelor ce contin missing values cu astfel de metode ar putea influenta drastic datele de iesire. (De exemplu: se observa ca sunt inregistrari cu persoane ce castiga anual peste $50k, la workClass avand missingValue, daca am substitui o astfel de valoare lipsa cu never-worked am ajunge sa denaturam ponderile categoriei persoanelor ce nu au lucrat niciodata, acest lucru ducand la coruperea estimarilor finale)
"""
data.fillna('unknown', inplace=True)

In [91]:
# Pregateste y
y = data.loc[:, 'Income']
y = np.where(y == '<=50K', 0, 1).reshape(data.shape[0], 1)
y = pd.DataFrame(y, columns=['Income'])

In [92]:
# Pregateste X (+ One-hot encoding pe datele categoriale din data)
X = data['Age']

data['WorkClass'] = pd.Categorical(data['WorkClass'])
X_categ = pd.get_dummies(data['WorkClass'], prefix='wc')
X = pd.concat([X, X_categ], axis=1)

X = pd.concat([X, data['Fnlwgt']], axis=1)

data['Education'] = pd.Categorical(data['Education'])
X_categ = pd.get_dummies(data['Education'], prefix='ed')
X = pd.concat([X, X_categ], axis=1)

X = pd.concat([X, data['EducationNum']], axis=1)

data['MaritalStatus'] = pd.Categorical(data['MaritalStatus'])
X_categ = pd.get_dummies(data['MaritalStatus'], prefix='ms')
X = pd.concat([X, X_categ], axis=1)

data['Occupation'] = pd.Categorical(data['Occupation'])
X_categ = pd.get_dummies(data['Occupation'], prefix='oc')
X = pd.concat([X, X_categ], axis=1)

data['Relationship'] = pd.Categorical(data['Relationship'])
X_categ = pd.get_dummies(data['Relationship'], prefix='rl')
X = pd.concat([X, X_categ], axis=1)

data['Race'] = pd.Categorical(data['Race'])
X_categ = pd.get_dummies(data['Race'], prefix='rc')
X = pd.concat([X, X_categ], axis=1)

data['Sex'] = pd.Categorical(data['Sex'])
X_categ = pd.get_dummies(data['Sex'], prefix='sx')
X = pd.concat([X, X_categ], axis=1)

X = pd.concat([X, data.loc[:, 'CapitalGain':'HoursPerWeek']], axis=1)

data['NativeCountry'] = pd.Categorical(data['NativeCountry'])
X_categ = pd.get_dummies(data['NativeCountry'], prefix='nc')
X = pd.concat([X, X_categ], axis=1)

In [93]:
# Scaleaza datele
X_columns = X.columns
X_index = X.index

min_max_scaler = preprocessing.MinMaxScaler()
X = pd.DataFrame(min_max_scaler.fit_transform(X), columns=X_columns, index=X_index)

In [94]:
# Separarea setului de date
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

In [95]:
# Verifica daca sunt egal distribuite
print(Counter(y_train.iloc[:,0]))
print(Counter(y_test.iloc[:,0]))

Counter({0: 16475, 1: 5232})
Counter({0: 8245, 1: 2609})


## Model 1: KNeighborsClassifier

In [97]:
# Model
"""
"""
model1 = KNeighborsClassifier(n_neighbors=2)
model1.fit(X_train, y_train)
y_hat = model1.predict(X_test)

In [98]:
print(f'Predicted:\n{y_hat}\n')
print(f'Ground truth:\n{np.array(y_test.iloc[:, 0])}\n')
print(f'Failed:\t{sum(y_hat != np.array(y_test.iloc[:, 0]))}')

Predicted:
[0 0 1 ... 0 0 0]

Ground truth:
[0 0 0 ... 0 0 1]

Failed:	2098
